In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas

import time

import pandas as pd
pd.set_option('display.max_columns', None)   
pd.set_option('display.max_rows', None)

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()

import warnings
warnings.filterwarnings('ignore')

opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False   # si True, no aparece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_extension('driver/adblock.crx')       # adblocker

opciones.add_argument('user-data-dir=selenium')    # mantiene las coockies

import Utils
from datetime import date

from decouple import config
user_spoty = config('user_spoty', default='')
pass_spoty = config('pass_spoty', default='')

from joblib import Parallel, delayed


In [2]:
autores1=pd.read_csv('../data/autores.csv')
autores2=pd.read_csv('../data/autores_2.csv')
autores3=pd.read_csv('../data/autores_3.csv')
lst=[autores1,autores2,autores3]
autores=pd.concat(lst)

In [22]:
csvs=['2017','2018','2019','2020','2021','2022']
autores=[]
lst_autores=[]
tmp=[]

import re

for i in csvs:
    lista_globa_spoty=pd.read_csv(f'../data/lista{i}.csv')
    for autor in lista_globa_spoty.artist_names:
    
        autor.strip()
        palabra=autor.split(',')
    
    
        for i in range(len(palabra)):
            autores.append(palabra[i])
        
    lst_autores.append(autores)

In [23]:
lst_prueba=[]
lst_prueba.extend(lst_autores[0])
lst_prueba.extend(lst_autores[1])
lst_prueba.extend(lst_autores[2])
lst_prueba.extend(lst_autores[3])
lst_prueba.extend(lst_autores[4])
lst_prueba.extend(lst_autores[5])

len(lst_prueba)

4264248

In [24]:
lst_prueba=list(set(lst_prueba))

In [25]:
len(lst_prueba)

2915

In [19]:
uniooon=pd.DataFrame(lst_prueba, columns=['autor'])

In [ ]:
uniooon.head()

In [21]:
uniooon.to_excel('../data/autores.xls',index=False,index_label=False)

In [41]:
def extraer_autores(autor):
    
    driver=webdriver.Chrome(PATH)
    driver.get('https://everynoise.com/engenremap.html')
    
    time.sleep(2)
    iframe=driver.find_element(By.XPATH, '/html/body/iframe')

    driver.switch_to.frame(iframe)
    
    input_autor=driver.find_element(By.XPATH, '/html/body/form/input[1]')
    
    time.sleep(2)
    
    input_autor.send_keys(str(autor))  # escribe texto

    input_autor.send_keys(u'\ue007')  # tecla ENTER

    time.sleep(2)
    
    datos=driver.find_element(By.XPATH,'/html/body/div').text
    
    
    
    
    cadena_final=datos
    
    #cadena_final=pd.DataFrame(cadena_final)
    
    driver.quit()
    
    return cadena_final

In [53]:
driver=webdriver.Chrome(PATH)
driver.get('https://everynoise.com/engenremap.html')
    
time.sleep(2)
iframe=driver.find_element(By.XPATH, '/html/body/iframe')

driver.switch_to.frame(iframe)
ttt=[]
for i in lst_prueba[670:]:
    
    ttt.append(extraer_autores(i))

In [46]:
len(ttt)

670

In [48]:
uniooon=pd.DataFrame(columns=['autor','estilo'])
for a,e in zip(lst_prueba[:671],ttt):
    uniooon=uniooon.append({'autor':a, 'estilo':e},ignore_index=True)

In [49]:
uniooon.to_csv('../data/autores_1.csv',index=False,index_label=False)